<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Image-Recongnition-Case-with-CNN-By-Tensorflow" data-toc-modified-id="Image-Recongnition-Case-with-CNN-By-Tensorflow-0.1">Image Recongnition Case with CNN By Tensorflow</a></span><ul class="toc-item"><li><span><a href="#1.Loading-data" data-toc-modified-id="1.Loading-data-0.1.1">1.Loading data</a></span><ul class="toc-item"><li><span><a href="#1.1创建TFReord文件(create-TFRecord-file)" data-toc-modified-id="1.1创建TFReord文件(create-TFRecord-file)-0.1.1.1">1.1创建TFReord文件(create TFRecord file)</a></span></li><li><span><a href="#1.2-读取TFRecord文件" data-toc-modified-id="1.2-读取TFRecord文件-0.1.1.2">1.2 读取TFRecord文件</a></span></li></ul></li><li><span><a href="#2.Building-a-Inception-Module" data-toc-modified-id="2.Building-a-Inception-Module-0.1.2">2.Building a Inception Module</a></span><ul class="toc-item"><li><span><a href="#2.1-a-block-of-inception-network" data-toc-modified-id="2.1-a-block-of-inception-network-0.1.2.1">2.1 a block of inception network</a></span></li><li><span><a href="#2.2-Inception-V1-architecture" data-toc-modified-id="2.2-Inception-V1-architecture-0.1.2.2">2.2 Inception-V1 architecture</a></span></li><li><span><a href="#2.3-finetune_InceptionV1" data-toc-modified-id="2.3-finetune_InceptionV1-0.1.2.3">2.3 finetune_InceptionV1</a></span></li></ul></li><li><span><a href="#3.Alexnet-model" data-toc-modified-id="3.Alexnet-model-0.1.3">3.Alexnet model</a></span><ul class="toc-item"><li><span><a href="#3.1-define-network-layers" data-toc-modified-id="3.1-define-network-layers-0.1.3.1">3.1 define network layers</a></span></li><li><span><a href="#3.2-implementation-of-Alexnet-architecture" data-toc-modified-id="3.2-implementation-of-Alexnet-architecture-0.1.3.2">3.2 implementation of Alexnet architecture</a></span></li><li><span><a href="#3.3-finetune_alexnet" data-toc-modified-id="3.3-finetune_alexnet-0.1.3.3">3.3 finetune_alexnet</a></span></li></ul></li><li><span><a href="#4.-VGGNet-model" data-toc-modified-id="4.-VGGNet-model-0.1.4">4. VGGNet model</a></span><ul class="toc-item"><li><span><a href="#4.1-implementation-of-VGG-16-architecture" data-toc-modified-id="4.1-implementation-of-VGG-16-architecture-0.1.4.1">4.1 implementation of VGG-16 architecture</a></span></li><li><span><a href="#4.2-finetune_vggnet" data-toc-modified-id="4.2-finetune_vggnet-0.1.4.2">4.2 finetune_vggnet</a></span></li></ul></li></ul></li></ul></li></ul></div>

## Image Recongnition Case with CNN By Tensorflow
Refrence List
- Coursera course [Deep Learning Specialization](https://www.coursera.org/specializations/deep-learning) by Andrew Ng
- [An Introduction to Implementing Neural Networks using Tensorflow](https://www.analyticsvidhya.com/blog/2016/10/an-introduction-to-implementing-neural-networks-using-tensorflow/)
- [Deep Learning with Tensorflow](https://github.com/caicloud/tensorflow-tutorial)

- The arthitechture of Inception-v3 is an classic [Convolutional Neural Network](https://en.wikipedia.org/wiki/Convolutional_neural_network)
 - stacking the output layers convoluting by different filter 
 - the different filter can detect various scale feature 
 - keeping scale invariance property
- Tensorflow is a low level library
- To define a neural network architecture, you should have fundamental knowledge of NN:
 - initialize parameter
 - forward propagation
 - compute cost
 - backward propagation
 - update parameter

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.python.framework import ops

from sklearn.model_selection import train_test_split
import os
path = r"F:\Diverse\statistics\Python_data_analysis\Kaggle项目\20170926_MNIST\dataset"
os.chdir(path)

%matplotlib inline
np.random.seed(1)

### 1.Loading data
- create a TFRecord file
- loading data API
 - tf.data.dataset
 - tf.data.TFRecordDataset

#### 1.1创建TFReord文件(create TFRecord file)

#### 1.2 读取TFRecord文件

In [2]:
#解析方法2
def parser(example):
    features={"image_raw": tf.FixedLenFeature([],tf.string),
          "labels": tf.FixedLenFeature([],tf.int64)}
    feature = tf.parse_single_example(example,features)
    images = tf.decode_raw(feature["image_raw"],tf.int64)
    label = tf.cast(feature["labels"], tf.int8)
    return images,label

def get_data(batch, epoch):
    filename = r"mnist.tfrecords" #地址必须已经存在
    dataset = tf.data.TFRecordDataset([filename])
    maps = dataset.map(parser)
    batch_data = maps.repeat(epoch)
    batch_data = batch_data.batch(batch) 
    iterator =batch_data.make_one_shot_iterator() #iterator对象
    one_element = iterator.get_next() #tensor对象
    return one_element

'''
sess = tf.Session()
for i in range(2):
    image_data,labels_data=sess.run(one_element)
'''

'\nsess = tf.Session()\nfor i in range(2):\n    image_data,labels_data=sess.run(one_element)\n'

In [5]:
test = pd.read_csv("test.csv")

To reduce training time, I just pick up 15% example for fitting my model.

In [6]:
train_sample = train.sample(frac=0.15)
X_train_flatten = train_sample.iloc[:,1:].values
Y_train_onehot = convert_to_one_hot(train_sample.iloc[:,0].values,10)
X_test_flatten = test.values

In [7]:
X_train, X_dev, Y_train, Y_dev = train_test_split(X_train_flatten, Y_train_onehot, random_state=42)

In [8]:
X_train = X_train/255
X_dev = X_dev/255
X_test = X_test_flatten/255

In [9]:
print ("number of training examples = " + str(X_train.shape[1]))
print ("number of dev examples = " + str(X_dev.shape[1]))
print ("number of test examples = " + str(X_test.shape[1]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_dev shape: " + str(X_dev.shape))
print ("Y_dev shape: " + str(Y_dev.shape))
print ("X_test shape: " + str(X_test.shape))

number of training examples = 784
number of dev examples = 784
number of test examples = 784
X_train shape: (4725, 784)
Y_train shape: (4725, 10)
X_dev shape: (1575, 784)
Y_dev shape: (1575, 10)
X_test shape: (28000, 784)


### 4. VGGNet model
 - VGG-16, 16-layers
 - VGG-19, 19-layers

#### 4.1 implementation of VGG-16 architecture

In [25]:
def vgg_16(inputs, num_classes=1000, keep_prob=0.5,
          scope="vgg_16", fc_conv_padding="VALID"):
    with tf.variable_scope(scope,"vgg_16", [inputs]) as sc:
        end_points_collection = sc.original_name_scope + "_end_points"
        with slim.arg_scope([slim.conv2d, slim.fully_connected, slim.max_pool2d],
                           outputs_collections=end_points_collection):
            net = slim.repeat(inputs, 2, slim.conv2d, 64, [3,3], scope="conv1")
            net = slim.max_pool2d(net, [2,2], scope="pool1")
            net = slim.repeat(net, 2, slim.conv2d, 128, [3,3], scope="conv2")
            net = slim.max_pool2d(net, [2,2], scope="pool2")
            net = slim.repeat(net, 3, slim.conv2d, 256, [3,3], scope="conv3")
            net = slim.max_pool2d(net, [2,2], scope="pool3")
            net = slim.repeat(net, 3, slim.conv2d, 512, [3,3], scope="conv4")
            net = slim.max_pool2d(net, [2,2], scope="pool4")
            net = slim.repeat(net, 3, slim.conv2d, 512, [3,3], scope="conv5")
            net = slim.max_pool2d(net, [2,2], scope="pool5")
            # use conv2d instead of fully_connected layers
            net = slim.conv2d(net, 4096, [7,7], padding=fc_conv_padding, scope="fc6")
            net = slim.dropout(net, keep_prob, scope="dropout6")
            net = slim.conv2d(net, 4096, [1,1], scope="fc7")
            net = slim.dropout(net, keep_prob, scope="dropout7")
            net = slim.conv2d(net, num_classes, [1,1], activation_fn=None, 
                              normalizer_fn=None, scope="fc8")
            net = tf.squeeze(net, [1,2], name="fc8/squeeze")
            # convert end_points_collection into a end_point dict
            end_points = slim.utils.convert_collection_to_dict(end_points_collection)
            end_points[sc.name+"fc8"] = net
        return net, end_points

#### 4.2 finetune_vggnet

In [ ]:
# 图像尺寸28x28x1太小，不适合深层网络
tf.reset_default_graph()
batch_sizes = 128
learning_rate = 0.01
keep_prob = 0.5
num_epochs = 10
num_classes = 10

with tf.name_scope("inputs"):
    x = tf.placeholder(tf.float32, [batch_sizes, 28, 28, 1])
    y = tf.placeholder(tf.float32, [batch_sizes, num_classes])

with tf.variable_scope("Vgg_model"):
    fc8, end_points_dict = vgg_16(x, num_classes)

with tf.name_scope("loss"):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=fc8, labels=y)
    loss = tf.reduce_mean(cross_entropy)
    
tf.summary.scalar("loss", loss)    

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.minimize(loss)

with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.argmax(fc8,1),tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

tf.summary.scalar("accuracy", accuracy)

mergered_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter(r"c:\users\bingo\desktop\log")

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    next_batch = get_data(batch_sizes, num_epochs)
    writer.add_graph(sess.graph)
    for epoch in range(num_epochs):
        img_batch, label_batch = sess.run(next_batch)
        img_batch = np.reshape(img_batch, [-1, 28, 28, 1])
        label_batch = sess.run(tf.one_hot(label_batch, num_classes))
        _, summary = sess.run([train_op, mergered_summary],feed_dict = {x:img_batch, y:label_batch})
        writer.add_summary(summary, epoch)
    writer.close()